## Algoritmos Genéticos
> Técnica de Computação evolucionária que serve para busca de solução para problemas de otimização.

> Com inspiração em termos da biologia evolutiva, tais como a hereditaridade, seleção natural, cruzamento e mutação.

ALGORITMO GENÉTICO
No problema do Caixeiro Viajante, temos uma lista de cidades e as distâncias entre elas; o objetivo é encontrar o menor caminho possível que passe por todas as cidades uma única vez e retorne à cidade de origem.
Suponha que temos 5 cidades. O objetivo é encontrar o menor caminho que passe por todas as cidades uma única vez e retorne à cidade inicial. Cada indivíduo da população será representado por uma lista de inteiros (Genoma), onde cada número indica a ordem de visita das cidades. Por exemplo: indivíduo 07: [2, 0, 3, 1, 4]. Isso representa uma rota que começa pela cidade 2, depois vai para a 0, em seguida 3, 1 e termina na 4 antes de retornar à cidade inicial. Rotas que repetem cidades recebem uma penalidade.
Nesse exemplo, as funções de seleção, reprodução (cruzamento) e mutação são realizadas por uma biblioteca chamada DEAP. Ela fornece ferramentas prontas para aplicar algoritmos genéticos, como o torneio para selecionar os melhores indivíduos, cruzamento em um ponto para gerar novos filhos e mutação aleatória em partes da rota. Essas operações são registradas e executadas automaticamente durante a evolução da população.

In [ ]:
# Importação dos módulos
import random
import numpy

# Biblioteca
!pip install deap
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [ ]:
# Gera a matriz de distâncias entre as cidades
def graphTSP(numCities, minDist, maxDist):
    cities = numpy.zeros((numCities, numCities), dtype=int)
    for i in range(numCities):
        for j in range(numCities):
            if (j > i):  # Preenche apenas a parte superior da matriz com distâncias aleatórias
                cities[i, j] = random.randint(minDist, maxDist)
            elif (j < i):  # Copia para a parte inferior para manter simetria
                cities[i, j] = cities[j, i]
    return cities

In [ ]:
# Número mínimo de cidades: 5
numCities = 5

# Solicita ao usuário o número de cidades até ser maior que 4
while True:
    numCities = int(input('Digite o número de cidades: '))
    if (numCities > 4):
        break
    else:
        print('O número de cidades deve ser maior que 4!')

# Cria o grafo com distâncias aleatórias entre as cidades
cities = graphTSP(numCities, 10, 100)
print('Grafo:\n', cities)

In [ ]:
# Definição da estrutura dos indivíduos e da população
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar o custo total
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

# Gera inteiros aleatórios representando cidades
toolbox.register("attr_int", random.randint, 0, numCities - 1)

# Gera um indivíduo com uma sequência de cidades
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, numCities)

# Gera uma população de indivíduos
toolbox.register("population", list, toolbox.individual)

In [ ]:
# Avalia o custo de uma rota (penaliza repetições de cidades)
def evalRoute(individual):
    cost = 0
    for i in range(1, len(individual)):
        if (individual.count(individual[i]) > 1):
            cost += 1000000  # Penalidade para cidades repetidas
        cost += cities[individual[i - 1], individual[i]]  # Soma a distância entre as cidades
    cost += cities[individual[i], individual[0]]  # Fecha o ciclo voltando à cidade inicial
    return (cost,)  # Retorna como tupla para compatibilidade com DEAP

In [ ]:
# Registro das funções para o algoritmo genético
toolbox.register("evaluate", evalRoute)  # Função de avaliação
toolbox.register("select", tools.selTournament, tournsize=3)  # Seleção por torneio
toolbox.register("mate", tools.cxOnePoint)  # Cruzamento por um ponto
toolbox.register("mutate", tools.mutUniformInt, low=0, up=numCities - 1, indpb=0.05)  # Mutação uniforme

# Função principal
def main():
    print('Execução do algoritmo genético:')

    random.seed(42)  # Define a semente para resultados reproduzíveis

    # Parâmetros da evolução
    NGEN = 100      # Número de gerações
    MU = 50         # Tamanho da população
    LAMBDA = 100    # Número de filhos gerados por geração
    CXPB = 0.7      # Probabilidade de cruzamento
    MUTPB = 0.3     # Probabilidade de mutação

    # Gera a população inicial
    pop = toolbox.population(n=MU)

    # Hall da fama para guardar os melhores indivíduos
    hof = tools.ParetoFront()

    # Estatísticas para acompanhar o progresso
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    # Executa o algoritmo genético usando o modelo (μ + λ)
    algorithms.eaMuPlusLambda(
        pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
        halloffame=hof
    )

    # Exibe a melhor rota encontrada e seu custo
    print('\nRota:', hof[0], '\nCusto:', evalRoute(hof[0])[0])
    return pop, stats, hof

# Ponto de entrada do programa
if __name__ == "__main__":
    main()